In [33]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

True

## Wikipedia Retriever

In [2]:
from langchain.retrievers import WikipediaRetriever

In [3]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2,lang="en")

In [4]:
# Define your query
query = "the geopolitical history of india and pakistan from the perspective of a america"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [5]:
docs

[Document(metadata={'title': 'Dominion of Pakistan', 'summary': 'The Dominion of Pakistan, officially Pakistan, was an independent federal dominion in the British Commonwealth of Nations, which existed from 14 August 1947 to 23 March 1956. It was created by the passing of the Indian Independence Act 1947 by the British parliament, which also created an independent Dominion of India.\nThe new dominion consisted of those presidencies and provinces of British India which were allocated to it in the Partition of India. Until 1947, these regions had been ruled by the United Kingdom as a part of the British Empire.\nIts status as a federal dominion within the British Empire ended in 1956 with the completion of the Constitution of Pakistan, which established the country as a republic. The constitution also administratively split the nation into West Pakistan and East Pakistan. Until then, these provinces had been governed as a singular entity, despite being separate geographic exclaves. Event

In [6]:
# Print retrieved content
for i,doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content} ---")


--- Result 1 ---
Content:
The Dominion of Pakistan, officially Pakistan, was an independent federal dominion in the British Commonwealth of Nations, which existed from 14 August 1947 to 23 March 1956. It was created by the passing of the Indian Independence Act 1947 by the British parliament, which also created an independent Dominion of India.
The new dominion consisted of those presidencies and provinces of British India which were allocated to it in the Partition of India. Until 1947, these regions had been ruled by the United Kingdom as a part of the British Empire.
Its status as a federal dominion within the British Empire ended in 1956 with the completion of the Constitution of Pakistan, which established the country as a republic. The constitution also administratively split the nation into West Pakistan and East Pakistan. Until then, these provinces had been governed as a singular entity, despite being separate geographic exclaves. Eventually, the East became Bangladesh and th

## Vector Store Retriever

In [7]:
from langchain.vectorstores import Chroma

# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [8]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="chroma_db",
    collection_name="my_collection"
)

In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [10]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [11]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.


In [12]:
results = vectorstore.similarity_search(query=query,k=2)

In [13]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.


## MMR

In [14]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
# Initialize Gemini embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model,
)

In [16]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3,"lambda_mult": 0.5}
)

In [17]:
query = "What is langchain?"
results = retriever.invoke(query)

In [18]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


## Multiquery Retriever

In [19]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [20]:
# Initialize Gemini embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model,
)

In [21]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [22]:
multiquery_retriever =MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")
)

In [23]:
# Query
query = "How to improve energy levels and maintain balance?"

In [24]:
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [25]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
MMR helps you get diverse results when doing similarity search.

--- Result 2 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 3 ---
Embeddings are vector representations of text.

--- Result 4 ---
LangChain is used to build LLM based applications.

--- Result 5 ---
Chroma is used to store and search document embeddings.
******************************************************************************************************************************************************

--- Result 1 ---
Embeddings are vector representations of text.

--- Result 2 ---
MMR helps you get diverse results when doing similarity search.

--- Result 3 ---
Chroma is used to store and search document embeddings.

--- Result 4 ---
LangChain makes it easy to work with LLMs.

--- Result 5 ---
LangChain is used to build LLM based applications.

--- Result 6 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


## ContextualCompressionRetriever

In [26]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [27]:
# Initialize Gemini embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model,
)

In [28]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [29]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")
compressor  = LLMChainExtractor.from_llm(llm=llm)

In [30]:
compression_retriever  = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [31]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [32]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis does not occur in animal cells.

--- Result 2 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
